[![Open in Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/fat-forensics/resources/master?filepath=pi_ice_pd%2Fpi_ice_pd.ipynb)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fat-forensics/resources/blob/master/pi_ice_pd/pi_ice_pd.ipynb)
[![new BSD](https://img.shields.io/github/license/fat-forensics/resources.svg)](https://github.com/fat-forensics/resources/blob/master/LICENCE)

# Interactive Exploration of Permutation Importance, Individual Conditional Expectation and Partial Dependence #

This directory contains a Jupyter Notebook that explores
*permutation importance*, *individual conditional expectation* and
*partial dependence* with interactive widgets.

To run the notebook (`pi_ice_pd.ipynb`) you need to install a
collection of Python dependencies listed in the `requirements.txt` file
(included in this directory) by executing `pip install -r requirements.txt`.
Alternatively, you can run it via Binder or Colab by clicking the buttons
included above.

In [1]:
#@title Install dependencies
# NBVAL_IGNORE_OUTPUT
import os
if 'google.colab' in str(get_ipython()):
    LIBRARY_URL = ('https://raw.githubusercontent.com/fat-forensics/resources/'
                   'master/pi_ice_pd/{}')

    require = 'requirements.txt'
    require_url = LIBRARY_URL.format(require)
    if not os.path.exists(require):
        ! wget $require_url -O $require
        ! pip install -r $require

    module = 'pi_ice_pd.py'
    module_url = LIBRARY_URL.format(module)
    if not os.path.exists(module):
        ! wget $module_url -O $module

In [2]:
#@title Set up dependencies
import numpy as np
import sklearn.datasets
import sklearn.linear_model

import pi_ice_pd as pid

22-Jul-08 20:37:33 fatf.utils.array.tools INFO     Using numpy's numpy.lib.recfunctions.structured_to_unstructured as fatf.utils.array.tools.structured_to_unstructured and fatf.utils.array.tools.structured_to_unstructured_row.


In [3]:
#@title Get iris data set and fit "black box"
# Fix global options
RANDOM_SEED = 42

# Load the iris data set
iris = sklearn.datasets.load_iris()
iris_X = iris.data  # [:, :2]  #  take the first two features only
iris_y = iris.target
iris_lables = iris.target_names
iris_feature_names = iris.feature_names

# Fit the classifier
logreg = sklearn.linear_model.LogisticRegression(
    C=1e5, random_state=RANDOM_SEED)
logreg.fit(iris_X, iris_y)

LogisticRegression(C=100000.0, random_state=42)

## Permutation Importance ##

In [4]:
#@title Widget specification
# Fix global options
PI_REPEATS = 30

pi_metrics = {
    'R-squared': 'r2',
    'MSE': 'neg_mean_squared_error',
    'MAE': 'neg_mean_absolute_error',
    'max': 'max_error'
}
feature_grouping = [[0, 1], [2, 3]]

In [5]:
#@title Generate permutation importance
pi = pid.build_permutation_importance(
    iris_X,
    iris_y,
    iris_feature_names,
    logreg,
    pi_metrics.values(),
    repeats=PI_REPEATS,
    random_seed=RANDOM_SEED
)

In [6]:
#@title Generate widget
# NBVAL_IGNORE_OUTPUT
pi_widget = pid.generate_pi_widget(
    pi_metrics, pi, feature_grouping, iris_X, iris_y, iris_feature_names)

In [7]:
#@title Display widget
# NBVAL_IGNORE_OUTPUT
pi_widget

## Individual Conditional Expectation & Partial Dependence ##

In [8]:
#@title Widget specification
# Fix global options
ICE_PD_SAMPLES = 200

feature_idxs = [2, 3]

class_labels = {i: j for i, j in enumerate(iris_lables)}
instances_to_explain = {
    'setosa': np.array([5, 3.5, 1.5, 0.25]).astype(np.float32),
    'versicolor': np.array([5.5, 2.75, 4.5, 1.25]).astype(np.float32),
    'virginica': np.array([7, 3, 5.5, 2.25]).astype(np.float32)
}

In [9]:
#@title Generate individual conditional expectation & partial dependence
ice_pd = pid.build_ice_pd(
    iris_X,
    logreg,
    feature_idxs,
    samples_no=ICE_PD_SAMPLES)

In [10]:
#@title Generate plain widget
# NBVAL_IGNORE_OUTPUT
ice_pd_widget = pid.generate_ice_pd_widget(
    ice_pd, class_labels, iris_feature_names)

In [11]:
#@title Display plain widget
# NBVAL_IGNORE_OUTPUT
ice_pd_widget

In [12]:
#@title Generate complex widget
# NBVAL_IGNORE_OUTPUT
ice_pd_widget_ = pid.generate_ice_pd_widget(
    ice_pd,
    class_labels,
    iris_feature_names,
    instances_to_explain=instances_to_explain,
    model=logreg,
    show_discretisation=True)

In [13]:
#@title Display complex widget
# NBVAL_IGNORE_OUTPUT
ice_pd_widget_